In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mariaherrerot/ddrdataset")

print("Path to dataset files:", path)

100%|██████████| 2.98G/2.98G [01:19<00:00, 40.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mariaherrerot/ddrdataset/versions/1


In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Model, regularizers, callbacks, mixed_precision
from tensorflow.keras.applications import ResNet50, VGG19
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

# Enable mixed precision
mixed_precision.set_global_policy('mixed_float16')

# Paths & params
BASE_DIR      = "/root/.cache/kagglehub/datasets/mariaherrerot/ddrdataset/versions/1"
CSV_PATH      = os.path.join(BASE_DIR, "DR_grading.csv")
IMG_DIR       = os.path.join(BASE_DIR, "DR_grading", "DR_grading")
IMG_SIZE      = 224
BATCH_SIZE    = 64
INITIAL_LR    = 1e-4
FINE_LR       = 1e-6
INITIAL_EPOCHS= 15
FINE_EPOCHS   = 5
AUTOTUNE      = tf.data.AUTOTUNE
WEIGHT_DECAY  = 1e-4

# Load & standardize CSV
raw = pd.read_csv(CSV_PATH)
for c in ['image','filename','ID']:
    if c in raw.columns:
        raw.rename(columns={c:'id_code'}, inplace=True)
        break
for c in ['level','label','Label']:
    if c in raw.columns:
        raw.rename(columns={c:'diagnosis'}, inplace=True)
        break
raw['id_code'] = raw['id_code'].str.strip().str.replace(r'\.jpg$', '', regex=True)
# map image paths
paths = glob.glob(os.path.join(IMG_DIR, "*.jpg"))
path_map = {os.path.splitext(os.path.basename(p))[0]: p for p in paths}
raw['image_path'] = raw['id_code'].map(path_map)
df = raw.dropna(subset=['image_path']).reset_index(drop=True)
NUM_CLASSES = df['diagnosis'].nunique()

# Split
train_val, test_df = train_test_split(df, test_size=0.10, stratify=df['diagnosis'], random_state=42)
val_ratio = 0.20 / 0.90
train_df, val_df = train_test_split(train_val, test_size=val_ratio,
                                     stratify=train_val['diagnosis'], random_state=42)

# Dataset builder
def build_ds(df, augment=False, shuffle=False, repeat=False):
    ds = tf.data.Dataset.from_tensor_slices((df['image_path'].values,
                                             df['diagnosis'].values))
    if shuffle: ds = ds.shuffle(len(df))
    if repeat:  ds = ds.repeat()
    def _parse(path, label):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
        img = tf.keras.applications.resnet.preprocess_input(img)
        return img, tf.one_hot(label, NUM_CLASSES)
    ds = ds.map(_parse, num_parallel_calls=AUTOTUNE)
    if augment:
        aug = tf.keras.Sequential([
            layers.RandomFlip('horizontal_and_vertical'),
            layers.RandomRotation(0.2),
            layers.RandomZoom(0.1, 0.1)
        ])
        ds = ds.map(lambda x, y: (aug(x, training=True), y), num_parallel_calls=AUTOTUNE)
    return ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

train_ds = build_ds(train_df, augment=True, shuffle=True, repeat=True)
val_ds   = build_ds(val_df)
test_ds  = build_ds(test_df)

# 1) ResNet50 model
def make_resnet_model():
    base = ResNet50(weights='imagenet', include_top=False,
                    input_shape=(IMG_SIZE,IMG_SIZE,3))
    base.trainable = False
    inp = layers.Input((IMG_SIZE,IMG_SIZE,3))
    x = base(inp, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(WEIGHT_DECAY))(x)
    x = layers.Dropout(0.5)(x)
    out = layers.Dense(NUM_CLASSES, activation='softmax', dtype='float32')(x)
    model = Model(inp, out)
    model.compile(optimizer=tf.keras.optimizers.Adam(INITIAL_LR),
                  loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
                  metrics=['accuracy'])
    return model

resnet_model = make_resnet_model()
resnet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=INITIAL_EPOCHS,
    steps_per_epoch=len(train_df)//BATCH_SIZE,
    callbacks=[callbacks.EarlyStopping('val_loss', patience=3, restore_best_weights=True)]
)
# Fine-tune last block
for layer in resnet_model.layers[1].layers[-50:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True
resnet_model.compile(optimizer=tf.keras.optimizers.Adam(FINE_LR),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])
resnet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=INITIAL_EPOCHS+FINE_EPOCHS,
    initial_epoch=INITIAL_EPOCHS,
    steps_per_epoch=len(train_df)//BATCH_SIZE,
    callbacks=[
        callbacks.ReduceLROnPlateau('val_loss',factor=0.5,patience=2),
        callbacks.EarlyStopping('val_loss',patience=3,restore_best_weights=True)
    ]
)

# 2) VGG19 feature extractor
def make_vgg_feat():
    base = VGG19(weights='imagenet', include_top=False,
                 input_shape=(IMG_SIZE,IMG_SIZE,3))
    base.trainable = False
    inp = layers.Input((IMG_SIZE,IMG_SIZE,3))
    x = base(inp, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    return Model(inp, x, name='vgg19_feat')

vgg_feat = make_vgg_feat()

# 3) Feature extraction
from tensorflow.keras.utils import to_categorical

def extract_feats(model, df):
    ds = build_ds(df)
    steps = len(df)//BATCH_SIZE + 1
    feats = model.predict(ds, steps=steps, verbose=1)
    labels = df['diagnosis'].values
    return feats, labels

X_tr_rn, y_tr    = extract_feats(resnet_model, train_df)
X_tr_vg, _       = extract_feats(vgg_feat, train_df)
X_val_rn, y_val  = extract_feats(resnet_model, val_df)
X_val_vg, _      = extract_feats(vgg_feat, val_df)
X_test_rn, y_test= extract_feats(resnet_model, test_df)
X_test_vg, _     = extract_feats(vgg_feat, test_df)

# Stack features for SVM
X_tr = np.hstack([X_tr_rn, X_tr_vg])
X_val= np.hstack([X_val_rn, X_val_vg])
X_te = np.hstack([X_test_rn, X_test_vg])

# 4) Train SVM on stacked features
svm = make_pipeline(StandardScaler(), SVC(kernel='rbf', probability=True, C=10))
svm.fit(X_tr, y_tr)
print("SVM val acc:", svm.score(X_val, y_val))

# 5) Build meta-features from all three models
p_tr_rn = resnet_model.predict(build_ds(train_df), steps=len(train_df)//BATCH_SIZE+1)
p_tr_vg = vgg_feat.predict(build_ds(train_df), steps=len(train_df)//BATCH_SIZE+1)
p_tr_vg = tf.keras.activations.softmax(layers.Dense(NUM_CLASSES, dtype='float32')(p_tr_vg)).numpy()
p_tr_svm= svm.predict_proba(X_tr)
meta_tr = np.hstack([p_tr_rn, p_tr_vg, p_tr_svm])

p_val_rn= resnet_model.predict(build_ds(val_df), steps=len(val_df)//BATCH_SIZE+1)
p_val_vg= vgg_feat.predict(build_ds(val_df), steps=len(val_df)//BATCH_SIZE+1)
p_val_vg= tf.keras.activations.softmax(layers.Dense(NUM_CLASSES, dtype='float32')(p_val_vg)).numpy()
p_val_svm= svm.predict_proba(X_val)

meta_val= np.hstack([p_val_rn, p_val_vg, p_val_svm])

# 6) Meta-learner: Logistic Regression
meta_clf = LogisticRegression(multi_class='multinomial', max_iter=2000)
meta_clf.fit(meta_tr, y_tr)
print("Stacked ensemble test acc:", meta_clf.score(np.hstack([resnet_model.predict(build_ds(test_df), steps=len(test_df)//BATCH_SIZE+1),tf.keras.activations.softmax(layers.Dense(NUM_CLASSES, dtype='float32')(vgg_feat.predict(build_ds(test_df), steps=len(test_df)//BATCH_SIZE+1))).numpy(),svm.predict_proba(X_te)]),y_test))


Epoch 1/15
136/136 ━━━━━━━━━━━━━━━━━━━━ 56s 292ms/step - accuracy: 0.4152 - loss: 1.9933 - val_accuracy: 0.6228 - val_loss: 1.1332
Epoch 2/15
136/136 ━━━━━━━━━━━━━━━━━━━━ 33s 244ms/step - accuracy: 0.5400 - loss: 1.3379 - val_accuracy: 0.6295 - val_loss: 1.1553
Epoch 3/15
136/136 ━━━━━━━━━━━━━━━━━━━━ 33s 243ms/step - accuracy: 0.5831 - loss: 1.2226 - val_accuracy: 0.6327 - val_loss: 1.1671
Epoch 4/15
136/136 ━━━━━━━━━━━━━━━━━━━━ 33s 241ms/step - accuracy: 0.5921 - loss: 1.2040 - val_accuracy: 0.6248 - val_loss: 1.1523
Epoch 16/20
136/136 ━━━━━━━━━━━━━━━━━━━━ 61s 289ms/step - accuracy: 0.5415 - loss: 1.2180 - val_accuracy: 0.6228 - val_loss: 1.0042 - learning_rate: 1.0000e-06
Epoch 17/20
136/136 ━━━━━━━━━━━━━━━━━━━━ 33s 244ms/step - accuracy: 0.5572 - loss: 1.1520 - val_accuracy: 0.6303 - val_loss: 0.9930 - learning_rate: 1.0000e-06
Epoch 18/20
136/136 ━━━━━━━━━━━━━━━━━━━━ 34s 251ms/step - accuracy: 0.5775 - loss: 1.1030 - val_accuracy: 0.6311 - val_loss: 0.9843 - learning_rate: 1.0000e

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step
Stacked ensemble test acc: 0.7581803671189146
